# Train

In [13]:
fig, ax = plt.subplots()

FigureCanvasNbAgg()

In [345]:
model = build_model()
trained_model = build_model()
trained_model.set_weights(model.get_weights())
#model.summary()

In [332]:
trained_model.set_weights(model.get_weights())

In [346]:
reset_samples()
eval_scores = [eval(trained_model, eval_cache_input, eval_cache_output)]
all_eval_scores = eval_scores[:]


In [368]:
reset_samples()

In [372]:
for i in range(20):  
    mcts_generate()
    clear_output()
    
    p = np.random.permutation(int(number_of_samples))
    if len(p) > BATCH_SIZE * TRAINING_LOOPS:
        p = p[:BATCH_SIZE * TRAINING_LOOPS]
    train_outputs = []
    train_inputs = []
    for i in range(len(sample_outputs)):
        train_outputs.append(sample_outputs[i][p])
    for i in range(len(sample_inputs)):
        train_inputs.append(sample_inputs[i][p])
        
    trained_model.fit(train_inputs, train_outputs, epochs=1, batch_size=BATCH_SIZE)    
    
    eval_new = eval(trained_model, eval_cache_input, eval_cache_output)
    if eval_new > eval_scores[-1]:
        eval_scores.append(eval_new)
        print("New model score: " + str(eval_scores[-1]) + "!")
        model.set_weights(trained_model.get_weights())
    print("Last scores: " + str(eval_scores[::-1]) + "!")
    all_eval_scores.append(eval_new)
    
plt.plot(all_eval_scores, label='6x75')
ax.legend(shadow=True)
plt.show()

Epoch 1/1
11646/11646 [==============================] - 0s 37us/step - loss: 0.1354 - dense_44_loss: 0.0017 - dense_48_loss: 0.1337 - dense_44_acc: 0.7254 - dense_48_acc: 0.5145


  0%|          | 0/375 [00:00<?, ?it/s]

Last scores: [0.72, 0.68, 0.67, 0.65, 0.64, 0.61, 0.51, 0.42, 0.29, 0.27, 0.17, 0.16, 0.15, 0.13]!


  2%|▏         | 8/375 [00:00<00:51,  7.16it/s]


KeyboardInterrupt: 

In [357]:
finished_sample_numbers

[1281, 1281, 1285, 2384, 1281]

In [354]:
train_outputs[1][0]

array([-0.33333333])

In [61]:
model.set_weights(trained_model.get_weights())

In [59]:
for i in range(3):
    trained_model.fit(sample_inputs, sample_outputs, epochs=1, batch_size=BATCH_SIZE)  
    print(eval(trained_model, eval_cache_input, eval_cache_output))

Epoch 1/1
300000/300000 [==============================] - 10s 32us/step - loss: 0.0921 - dense_12_loss: 9.4333e-04 - dense_16_loss: 0.0911 - dense_12_acc: 0.3069 - dense_16_acc: 0.9478
0.85
Epoch 1/1
300000/300000 [==============================] - 10s 34us/step - loss: 0.0907 - dense_12_loss: 9.4048e-04 - dense_16_loss: 0.0898 - dense_12_acc: 0.3075 - dense_16_acc: 0.9481
0.85
Epoch 1/1
300000/300000 [==============================] - 12s 39us/step - loss: 0.0910 - dense_12_loss: 9.3987e-04 - dense_16_loss: 0.0901 - dense_12_acc: 0.3068 - dense_16_acc: 0.9482
0.8


In [333]:
eval(trained_model, eval_cache_input, eval_cache_output)

0.83

In [61]:
model_output = model.predict([sample_inputs[0], sample_inputs[1]])[0]
a = np.argmax(model_output, axis=-1)
k = np.equal(np.take(sample_outputs[0], np.argmax(sample_outputs[0], axis=-1)), np.take(sample_outputs[0], a)).sum()
k / len(sample_outputs[0])

0.9862666666666666

In [53]:
model.save('model.h5')

In [15]:
model.load_weights('modelDense.h5')

In [62]:
compare(model, None)

0.35

In [59]:
match([model, None])

1